In [6]:
import ray
import numpy as np

In [2]:
ray.init()

2023-10-17 06:50:46,349	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.11.179:6379...
2023-10-17 06:50:46,362	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 192.168.11.179:8265 


Python version:,3.9.18
Ray version:,2.7.1
Dashboard:,http://192.168.11.179:8265


In [3]:
@ray.remote(num_gpus=2)
def f():
    print(ray.get_gpu_ids())

In [4]:
result = f.remote()
result

ObjectRef(c8ef45ccd0112571ffffffffffffffffffffffff0100000001000000)

(f pid=1312) [0, 1]


In [ ]:
ray.get(result)

In [9]:
@ray.remote
def add(a, b):
    return np.add(a, b)

In [19]:
@ray.remote
def create_array():
    return np.ones(3)

In [20]:
a1 = create_array.remote()
a2 = create_array.remote()
sum = add.remote(a1, a2)
sum

ObjectRef(c54e76759b2a0c10ffffffffffffffffffffffff0100000001000000)

In [22]:
ray.get(sum)

array([2., 2., 2.])

In [9]:
import os
from filelock import FileLock
from typing import Dict

import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor, Normalize
from tqdm import tqdm

import ray.train
from ray.train import ScalingConfig
from ray.train.torch import TorchTrainer


def get_dataloaders(batch_size):
    # Transform to normalize the input images
    transform = transforms.Compose([ToTensor(), Normalize((0.5,), (0.5,))])

    with FileLock(os.path.expanduser("~/data.lock")):
        # Download training data from open datasets.
        training_data = datasets.FashionMNIST(
            root="~/data",
            train=True,
            download=True,
            transform=transform,
        )

        # Download test data from open datasets.
        test_data = datasets.FashionMNIST(
            root="~/data",
            train=False,
            download=True,
            transform=transform,
        )

    # Create data loaders.
    train_dataloader = DataLoader(training_data, batch_size=batch_size)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    return train_dataloader, test_dataloader


# Model Definition
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Dropout(0.25),
            nn.Linear(512, 10),
            nn.ReLU(),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


def train_func_per_worker(config: Dict):
    lr = config["lr"]
    epochs = config["epochs"]
    batch_size = config["batch_size_per_worker"]

    # Get dataloaders inside worker training function
    train_dataloader, test_dataloader = get_dataloaders(batch_size=batch_size)

    # [1] Prepare Dataloader for distributed training
    # Shard the datasets among workers and move batches to the correct device
    # =======================================================================
    train_dataloader = ray.train.torch.prepare_data_loader(train_dataloader)
    test_dataloader = ray.train.torch.prepare_data_loader(test_dataloader)

    model = NeuralNetwork()

    # [2] Prepare and wrap your model with DistributedDataParallel
    # Move the model the correct GPU/CPU device
    # ============================================================
    model = ray.train.torch.prepare_model(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9)

    # Model training loop
    for epoch in range(epochs):
        model.train()
        for X, y in tqdm(train_dataloader, desc=f"Train Epoch {epoch}"):
            pred = model(X)
            loss = loss_fn(pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        test_loss, num_correct, num_total = 0, 0, 0
        with torch.no_grad():
            for X, y in tqdm(test_dataloader, desc=f"Test Epoch {epoch}"):
                pred = model(X)
                loss = loss_fn(pred, y)

                test_loss += loss.item()
                num_total += y.shape[0]
                num_correct += (pred.argmax(1) == y).sum().item()

        test_loss /= len(test_dataloader)
        accuracy = num_correct / num_total

        # [3] Report metrics to Ray Train
        # ===============================
        ray.train.report(metrics={"loss": test_loss, "accuracy": accuracy})


def train_fashion_mnist(num_workers=2, use_gpu=False, epochs=10):
    global_batch_size = 32

    train_config = {
        "lr": 1e-3,
        "epochs": epochs,
        "batch_size_per_worker": global_batch_size // num_workers,
    }

    # Configure computation resources
    scaling_config = ScalingConfig(num_workers=num_workers, use_gpu=use_gpu)

    # Initialize a Ray TorchTrainer
    trainer = TorchTrainer(
        train_loop_per_worker=train_func_per_worker,
        train_loop_config=train_config,
        scaling_config=scaling_config,
    )

    # [4] Start Distributed Training
    # Run `train_func_per_worker` on all workers
    # =============================================
    result = trainer.fit()
    print(f"Training result: {result}")

In [10]:
train_fashion_mnist(num_workers=1, use_gpu=True, epochs=5)

(TorchTrainer pid=21207) Starting distributed worker processes: ['21242 (192.168.10.82)']
(RayTrainWorker pid=21242) Setting up process group for: env:// [rank=0, world_size=1]


(RayTrainWorker pid=21242) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz
(RayTrainWorker pid=21242) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-images-idx3-ubyte.gz to /home/ubuntu/data/FashionMNIST/raw/train-images-idx3-ubyte.gz


100%|██████████| 26421880/26421880 [00:09<00:00, 2721920.74it/s]


(RayTrainWorker pid=21242) Extracting /home/ubuntu/data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /home/ubuntu/data/FashionMNIST/raw
(RayTrainWorker pid=21242) 
(RayTrainWorker pid=21242) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz
(RayTrainWorker pid=21242) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/train-labels-idx1-ubyte.gz to /home/ubuntu/data/FashionMNIST/raw/train-labels-idx1-ubyte.gz
(RayTrainWorker pid=21242) Extracting /home/ubuntu/data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /home/ubuntu/data/FashionMNIST/raw
(RayTrainWorker pid=21242) 
(RayTrainWorker pid=21242) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-images-idx3-ubyte.gz


100%|██████████| 29515/29515 [00:00<00:00, 697471.32it/s]


...

(RayTrainWorker pid=21242) Extracting /home/ubuntu/data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /home/ubuntu/data/FashionMNIST/raw
(RayTrainWorker pid=21242) 
(RayTrainWorker pid=21242) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz


100%|██████████| 4422102/4422102 [00:12<00:00, 354881.43it/s]


(RayTrainWorker pid=21242) Downloading http://fashion-mnist.s3-website.eu-central-1.amazonaws.com/t10k-labels-idx1-ubyte.gz to /home/ubuntu/data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz
(RayTrainWorker pid=21242) Extracting /home/ubuntu/data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /home/ubuntu/data/FashionMNIST/raw
(RayTrainWorker pid=21242) 


100%|██████████| 5148/5148 [00:00<00:00, 28675002.65it/s]
(RayTrainWorker pid=21242) Moving model to device: cuda:0
Test Epoch 4: 100%|██████████| 313/313 [00:02<00:00, 151.45it/s]
2023-10-16 19:27:37,034	INFO tune.py:1143 -- Total run time: 116.68 seconds (116.64 seconds for the tuning loop).


Training result: Result(
  metrics={'loss': 0.4014163756856141, 'accuracy': 0.8538},
  path='/home/ubuntu/ray_results/TorchTrainer_2023-10-16_19-25-40/TorchTrainer_cab2a_00000_0_2023-10-16_19-25-40',
  filesystem='local',
  checkpoint=None
)


In [11]:
train_fashion_mnist(num_workers=2, use_gpu=True, epochs=5)

(TorchTrainer pid=21297) Starting distributed worker processes: ['21331 (192.168.10.82)', '21332 (192.168.10.82)']
(RayTrainWorker pid=21331) Setting up process group for: env:// [rank=0, world_size=2]
(RayTrainWorker pid=21331) Moving model to device: cuda:0
(RayTrainWorker pid=21331) Wrapping provided model in DistributedDataParallel.
Test Epoch 4: 100%|██████████| 313/313 [00:01<00:00, 250.86it/s]
2023-10-16 19:28:46,611	INFO tune.py:1143 -- Total run time: 69.54 seconds (69.52 seconds for the tuning loop).


Training result: Result(
  metrics={'loss': 0.39381615475367626, 'accuracy': 0.8572},
  path='/home/ubuntu/ray_results/TorchTrainer_2023-10-16_19-27-37/TorchTrainer_10431_00000_0_2023-10-16_19-27-37',
  filesystem='local',
  checkpoint=None
)
